<a href="https://colab.research.google.com/github/wqw547243068/2018-MachineLearning-Lectures-ESA/blob/master/courses/chapter_3/big_bang_theory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Big Bang台词分析
- 【2019-05-20】王奇文，wqw547243068@163.com
### 生活大爆炸
- 《生活大爆炸》（英文：The Big Bang Theory 简称：TBBT）
- 《生活大爆炸》是CBS电视台最具风格的一部情景喜剧，该剧以越来越盛行的“宅文化”吸引了一大批固定收视人群，虽然在各大奖项上暂无作为，但《生活大爆炸》的火爆程度和逐年上升的收视率是有目共睹的
- 2019年5月1日，《生活大爆炸》最后一集宣布正式杀青

### 人物介绍

- 主要人物：
   - Leonard Hofstadter —约翰尼·盖尔克奇 饰
   - Sheldon Cooper —吉姆·帕森斯 饰
   - Penny—卡蕾·措科 饰
   - Howard Wolowitz —西蒙·赫尔伯格 饰
   - Raj Koothrappali —昆瑙·内亚 饰
   - Leslie Winkle —莎拉·吉尔伯特 饰
   - Mrs. Hofstadter—克里斯汀·芭伦斯基 饰
   - Bernadette—梅丽莎·劳奇 饰
   - Mrs.Wolowitz—卡萝尔·安·苏希 饰
   - Amy Farrah Fowler—Mayim Bialik 饰
   - Priya—Aarti Mann 饰
- ![](https://pmcvariety.files.wordpress.com/2019/05/the-big-bang-theory-artisans.jpg?w=1000&h=563&crop=1)

### 其它
- 官方台词[big bang theory transcripts](https://bigbangtrans.wordpress.com/)
- 台词数据数据获取[地址](https://github.com/skashyap7/TBBTCorpus)
- ![](https://timgsa.baidu.com/timg?image&quality=80&size=b9999_10000&sec=1558371710386&di=9a83fe8f4ed1302d9e341bb6dd50eaf9&imgtype=jpg&src=http%3A%2F%2Fimg1.imgtn.bdimg.com%2Fit%2Fu%3D20824956%2C3357565608%26fm%3D214%26gp%3D0.jpg)
- [生活大爆炸(TBBT)：台词爬取、词云生成与NLP分析](https://blog.csdn.net/Tele_Anti_Nomy/article/details/88092709)

## 数据准备
- 下载数据，将json格式加载进来
- 发现一共202篇台词

In [0]:
%%bash
# 数据来自：https://github.com/skashyap7/TBBTCorpus
#http_file='https://github.com/skashyap7/TBBTCorpus/raw/master/data/final.json'
http_file='https://github.com/skashyap7/TBBTCorpus/raw/master/preprocessing/corpus.json'
wget $http_file

Couldn't find program: 'bash'


In [0]:
%%bash
#rm corpus.json.* # 删除重复文件
du -sh * # 计算文件大小,42m

3.4M	corpus.json
55M	sample_data


In [0]:
import json
import sys
data_file = 'C:/Users/wqw/Desktop/corpus.json'
#data_file = 'corpus.json'
data_raw = json.load(open(data_file))
#data_raw = json.loads(open(data_file).readlines())
print(len(data_raw.keys()), data_raw.keys())
len(data_raw['1_1'])

202 dict_keys(['8_17', '7_23', '7_10', '3_7', '4_2', '8_1', '5_12', '7_6', '9_2', '3_2', '3_11', '2_19', '1_4', '2_2', '6_21', '6_22', '5_23', '4_18', '2_20', '4_14', '6_3', '4_12', '8_20', '5_20', '8_14', '5_17', '2_13', '3_18', '7_5', '5_13', '5_7', '2_6', '7_12', '1_12', '4_22', '5_4', '6_1', '4_21', '3_1', '9_1', '6_12', '6_6', '7_24', '4_24', '1_10', '1_6', '2_1', '2_23', '3_21', '2_11', '3_13', '1_3', '6_14', '5_15', '2_16', '9_19', '1_17', '5_18', '1_13', '6_19', '2_17', '9_3', '8_11', '8_21', '6_13', '7_18', '1_9', '5_2', '3_5', '5_11', '2_9', '2_4', '1_11', '9_6', '8_9', '8_22', '4_6', '5_16', '5_19', '4_13', '1_15', '4_9', '3_3', '5_8', '1_1', '4_7', '7_14', '3_14', '4_1', '8_6', '6_7', '5_6', '7_15', '3_19', '8_4', '1_7', '3_22', '7_13', '6_15', '5_21', '7_17', '2_5', '8_16', '2_21', '9_9', '7_2', '6_9', '3_9', '4_11', '8_12', '9_15', '6_4', '5_22', '1_5', '8_8', '2_14', '4_3', '9_17', '8_7', '6_2', '9_23', '2_7', '6_5', '2_15', '6_16', '8_19', '2_12', '9_4', '6_10', '7_11',

19670

## 计算台词数目
- 每个场景下，每个人物的台词数据处理
- 统计台词数目

In [0]:
import sys
#data_raw['8_9']
# 每个场景、每个人
sen_dict = {}
# 每个场景、人物的累计台词数
sen_count_dict = {'scene':{}, 'person':{}}
cnt = 0
for k, v in data_raw.items():
    cnt += 1
    if cnt > 2:
        #break
        pass
    # 读取每篇文章
    first_line = False
    for line in v.strip().split('\n'):
        # 场景提取：
        #  第一种情形（文本首句）：\n\t\t\t\tScene: The apartment.\nLeonard: The math is all there.
        #  第二种情形（文本中间）：Penny: Okay, look, you promised me you would keep my secret so you’re just going to have to figure out a way to do it.
        # Scene: The apartment.
        # Scene: Rothman’s office. Sheldon is trying to affix paper over a fiercely blowing air vent.
        # (Scene turns into a Bollywood musical), 'The stairwell, approaching the apartment door'
        scene_1 = '-'
        if line.find('Scene:') != -1:
            scene_1 = line.split(':')[1]
            scene = scene_1.split('.')[0].strip()
            if scene.find(',') != -1:
                scene = scene.split(',')[0]
            # 分情形处理
            if first_line:
                # 文本中间,再次出现
                pass
            else:
                # 文本首句，首次出现
                if len(scene) < 2:
                    continue
                first_line = True
                continue
        if line.find('Like this') != -1:
            # 剔除尾部无效对白
            continue
        # 提取对话模式， person: sentence
        arr = [i.strip() for i in line.strip().split(': ')]
        if len(arr) < 2:
            if arr and arr[0].strip():
                print('旁白:%s, %s, %s'%(len(arr), scene_1, line))
            continue
        person = arr[0]
        sen = ''.join(arr[1:])
        # 数据清洗，Sheldon (on video)， Like this
        if person.find(' (') != -1:
            person = person.split(' (')[0]
        if person.find('(') != -1:
            person = person.strip('(')[0]
        if not scene in sen_dict:
            sen_dict[scene] = {}
        # 聚合每个场景下的同一个人
        if not person in sen_dict[scene]:
            sen_dict[scene][person] = []
        sen_dict[scene][person].append(sen)
        sen_count_dict['scene'][scene] = sen_count_dict['scene'].get(scene, 0) + 1
        sen_count_dict['person'][person] = sen_count_dict['person'].get(person, 0) + 1
    print('summury: {}\t{}\t{}'.format(k, scene, arr))
print(sen_dict.keys())
print(sen_dict['The stairwell']['Leonard'][:10])
print(sen_count_dict['scene'])
print(sen_count_dict['person'])

旁白:1, -, Credits sequence.
summury: 8_17	Emily’s bedroom	['Emily', 'Don’t worry about it. Good night.']
旁白:1, -, Credits sequence.
summury: 7_23	Mrs Wolowitz’s house	['Howard', 'Welcome to Team Putz.']
旁白:1, -, Credits sequence.
旁白:1, -, Wil:Can I blow the whistle?
summury: 7_10	The radio studio	['Amy', 'I’ll get the vodka.']
旁白:1, -, Credits sequence
旁白:1, -, try and understand. Look, Leonard and I are in a relationship, and occasionally, we’re going to fight. But no matter what happens between us, we will always love you. Right, Leonard?
summury: 3_7	The apartment	['Sheldon', 'I should have asked for much more than a comic book and a robot.']
旁白:1, -, Credits sequence.
summury: 4_2	The hallway	['Penny', 'Soft kitty, warm kitty, little ball of fur. Happy kitty, sleepy kitty, purr, purr, purr.']
旁白:1, -, Following a “previously on” sequence.
旁白:1, -, Credits sequence. 
summury: 8_1	The apartment	['Sheldon', 'I know. You know, I almost died in a fire in Des Moines, but I stayed put. FYI

旁白:1, -, Siri, I’m in the mood for gelato.
summury: 5_14	An office suite	['Raj (waking from a dream)', 'No!']
旁白:1, -, Credits sequence.
summury: 9_22	The apartment	['Sheldon', 'Leonard, friends are like toilet paper. It’s good to have extras under the sink.']
旁白:1, -, Credits sequence.
旁白:1, -, From Penny’s apartment they hear Penny and Sheldon singing “Blow the man down.” They enter to find Sheldon and Penny working together on the barettes.
旁白:1, -, Time shift
旁白:1, -, Time shift
旁白:1, -, Time shift
summury: 2_18	The apartment	['Howard', 'Sorry, guys, but Secret Agent Laser Obstacle Lunch is just stupid.']
旁白:1, -, Credits sequence.
summury: 7_21	The apartment	['Sheldon', 'And how do you propose to do that? (Amy takes off coat and is dressed as a Catholic schoolgirl) Unless you have Gravity on Blu-ray under that skirt, I don’t know where you’re going with this.']
旁白:1, -, Credits sequence. 
旁白:1, -, actually playing. When we watch Star Wars, we don’t say, we defeated the Empire.
sum

## 台词数目整体可视化
- 场景台词分布
- 人物台词分布

In [0]:
import numpy as np
MAX = 20
category = 'scene'
name_list = list(sen_count_dict[category].keys())
freq_list = list(sen_count_dict[category].values())
a = np.array(name_list)
b = np.array(freq_list)
print(len(a), len(b)) # 一共485种取值，太多，截取top
idx = b.argsort()[-MAX:]
print(a[idx])
print(b[idx])

485 485
['The lobby' 'Howard and Bernadette’s house' 'A restaurant'
 'Leonard’s bedroom' 'The hallway' 'Howard’s bedroom' 'Sheldon’s bedroom'
 'The living room' 'Amy’s apartment' 'Leonard’s car'
 'The university cafeteria' 'The same' 'Raj’s apartment'
 'Howard and Bernadette’s apartment' 'The Cheesecake Factory'
 'The comic book store' 'The cafeteria' 'The stairwell' 'Penny’s apartment'
 'The apartment']
[  452   474   479   508   509   510   516   555   605   642   737   760
   767   824   892  1145  1477  1483  3490 12594]


In [0]:
import numpy as np
import pyecharts as pe

page = pe.Page('老友记台词整体分布')
category = 'scene'
MAX = 20 # 显示最多类目
for category in ['scene', 'person']:
    name_list = list(sen_count_dict[category].keys())
    freq_list = list(sen_count_dict[category].values())
    a = np.array(name_list)
    b = np.array(freq_list)
    idx = b.argsort()[-MAX:]
    name_list = a[idx]
    freq_list = b[idx]
    #chart = pe.WordCloud('Scene: %s'%(scene), subtitle='Person: %s'%(person), title_pos='center')
    chart1 = pe.Bar('台词整体分布-%s-Top %s'%(category, MAX), title_pos='center')
    chart1.add("", name_list, freq_list, mark_point=['max','min'], width=900, is_label_show=True)
    page.add(chart1)
    chart2 = pe.Pie('台词整体分布-%s-Top %s'%(category, MAX), title_pos='center')
    chart2.add("", name_list, freq_list, is_legend_show=False,is_label_show=True)
    page.add(chart2)
# 添加台词数分布
# bar = pe.Bar()
# name_list = person_count.keys()
# val_list = person_count.values()
# bar.add("", name_list, val_list)
# page.add(bar)
page.render('vis_summary.html')
page

[<pyecharts.charts.bar.Bar at 0x2a63ee3da58>,
 <pyecharts.charts.pie.Pie at 0x2a619c8ff60>,
 <pyecharts.charts.bar.Bar at 0x2a619c8ae10>,
 <pyecharts.charts.pie.Pie at 0x2a619c8aa20>]

In [0]:
for k,v in sen

## NLP提取关键词
- 句子分词，提取关键词
- 看看每个人物都爱说啥

In [0]:
# 每个场景、每个人分词
import jieba
from jieba import analyse
word_dict = {}
for s in sen_dict:
    if s not in word_dict:
        word_dict[s] = {}
    for p in sen_dict[s]:
        if p not in word_dict[s]:
            word_dict[s][p] = {}
        # 每个场景每个人说过的所有话，列表
        speak_list = sen_dict[s][p]
        for w in speak_list:
            # 提取关键词
            #seg_list = jieba.cut(text)
            #res = analyse.TFIDF(w) # tf-idf提取关键词
            res = analyse.extract_tags(w)
            #res = analyse.textrank(w) # text-rank提取关键词
            for word in res:
                word_dict[s][p][word] = word_dict[s][p].get(word, 0) + 1
print(json.dumps(word_dict, ensure_ascii=False, indent=2))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### 测试
- 查看效果，指定任意场景、人物，显示其常见词语

In [0]:
scene = 'The stairwell'
person = 'Leonard'
#word_dict.values()
#word_dict[scene][person]
sen_dict[scene][person][:10]

['I was unstoppable. I mean, I was, I was on fire. It was like my mind and my body were totally connected, like, like athletes must feel when they’re in the zone.',
 'Admit it, you’re a little turned on.',
 'Why not?',
 'Hey.',
 'Okay.',
 'Okay.',
 'Well, we’re, we’re very happy for you.',
 'Why a turtle?',
 'Well, congratulations. Who would’ve thought you two would be the first in our group to start a family?',
 'Yeah. If you had told anyone that you were going away on a train by yourself across the country, do you know what they would have said?']

## 人物台词可视化

In [0]:
%%bash
# 安装pyecharts，1.0以下版本
#!pip install pyecharts
pip search  pyecharts

pyecharts-javascripthon (0.0.6)        - Embeded Python functions in pyecharts
pyecharts-snapshot (0.2.0)             - renders pyecharts output as image
pyecharts-extras (0.0.1)               - Draw speciaized graphs using pyecharts
pyecharts-plus (0.1.7)                 - Some charts which pyecharts don't contain
pyecharts-jupyter-installer (0.0.3)    - Install pyecharts extensions into jupyter
pyecharts (1.1.0)                      - Python options, make charting easier
  INSTALLED: 1.1.0 (latest)
reactive-pyecharts (1.0.0)             - Python options, make charting easier
eplot (0.1.1)                          - pandas interface for pyecharts
snapshot-phantomjs (0.0.3)             - Render pyecharts as image via phantomjs
tradeplot (0.4)                        - use pyecharts to realize trading chart
echarts-china-provinces-pypkg (0.0.3)  - Python package for chinese provinces for pyecharts
echarts-countries-pypkg (0.1.6)        - pyecharts map extensions - world countries - pytho

In [0]:
# 1.0以上版本的代码
from pyecharts import options as opts
from pyecharts.charts import Gauge, Page

def gauge_base() -> Gauge:
    c = (
        Gauge()
        .add("", [("完成率", 66.6)])
        .set_global_opts(title_opts=opts.TitleOpts(title="Gauge-基本示例"))
    )
    return c
gauge_base().render_notebook('t.html')

TypeError: ignored

## wordcloud词云可视化
- 用wordcloud词云可视化
- 自带停用词过滤功能
- [TelAntiNomy](https://blog.csdn.net/Tele_Anti_Nomy/article/details/88092709)
- ![](https://img-blog.csdnimg.cn/20190305094504431.jpg?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L1RlbGVfQW50aV9Ob215,size_16,color_FFFFFF,t_70)
- ![](https://img-blog.csdnimg.cn/2019030509452696.jpg?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L1RlbGVfQW50aV9Ob215,size_16,color_FFFFFF,t_70)

In [0]:
!pip install wordcloud

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [0]:
import matplotlib.pyplot as plt
txtname = data_file
with open(txtname,'r', encoding='UTF-8') as f:
    mytext = f.read()  # 直接读取全部内容
imgname = 'https://timgsa.baidu.com/timg?image&quality=80&size=b9999_10000&sec=1558371710386&di=9a83fe8f4ed1302d9e341bb6dd50eaf9&imgtype=jpg&src=http%3A%2F%2Fimg1.imgtn.bdimg.com%2Fit%2Fu%3D20824956%2C3357565608%26fm%3D214%26gp%3D0.jpg'
back_coloring = plt.imread(imgname)
image_colors = ImageColorGenerator(back_coloring)
wc = WordCloud(background_color="white",  # 背景颜色
               max_words=2000,  # 词云显示的最大词数
               mask=back_coloring,  # 设置背景图片
               random_state=2,
               margin=2  # 词语边缘像素距离
               )
wc.generate(mytext)
plt.figure()
plt.imshow(wc.recolor(color_func=image_colors))
plt.axis("off")
plt.show()
wc.to_file(eptname)

ValueError: invalid PNG header

In [0]:
import pyecharts as pe

# scene = 'The stairwell'
# person = 'Leonard'
page = pe.Page('老友记词云可视化')
person_count = {}
for scene in word_dict:
    for person in word_dict[scene]:
        name_list = word_dict[scene][person].keys()
        freq_list = word_dict[scene][person].values()
        chart = pe.WordCloud('Scene: %s'%(scene), subtitle='Person: %s'%(person), title_pos='center')
        chart.add("", name_list, freq_list, word_size_range=[30, 100], rotate_step=66)
        page.add(chart)
        # 统计台词数
        person_count[person] = person_count.get(person, 0) + len(sen_dict[scene][person])
# 添加台词数分布
# bar = pe.Bar()
# name_list = person_count.keys()
# val_list = person_count.values()
# bar.add("", name_list, val_list)
# page.add(bar)
page.render('vis_wordcloud.html')
page

In [0]:
'The stairwell' in list(word_dict.keys())
'The stairwell' in word_dict.keys()

True

## 热力图可视化

In [0]:
import pyecharts as pe
from numpy import random

# scene = 'The stairwell'
# person = 'Leonard'
# page = pe.Page('老友记词云可视化') 
p_list = []
x_axis = [] # 所有场景
for i in word_dict:
    x_axis.append(i)
    p_list.extend(word_dict[i].keys())
print(p_list)
x_len = len(x_axis)
print('x: ',x_axis)
y_axis = list(set(p_list)) # 所有人物
y_len = len(y_axis)
print('y: ',y_axis)
print(type(x_axis), type(y_axis))
data = []
for i in range(x_len):
    val = 0
    #print('xx===>',x_axis[i], word_dict)
    if x_axis[i] not in word_dict:
        print('x===>',x_axis[i], word_dict.keys())
        val = 0
    #continue
    for j in range(y_len):
        if x_axis[i] not in word_dict:
            print('--->100', x_axis[i], word_dict.keys())
            pass
        elif y_axis[j] not in word_dict[x_axis[i]]:
            #print('y===>',x_axis[i], y_axis[i], word_dict[x_axis[i]].keys())
            val = 0
        else:
            val = len(word_dict[x_axis[i]][y_axis[j]]) # 单词频次
            #val = len(sen_dict[x_axis[i]][y_axis[j]]) # 说话频率
        data.append([j, i, val])
print(data)
#data = [[i, j, random.randint(0, 50)] for i in range(x_len) for j in range(y_len)]
heatmap = pe.HeatMap()
heatmap.add("Scene-Person Heatmap", y_axis, x_axis, data, is_visualmap=True,
            visual_text_color="#000", visual_orient='horizontal')
heatmap.render('heatmap_freq.html')
heatmap

['Leonard', 'Penny', 'Sheldon', 'Amy', 'Together', 'Scene', 'Raj', 'Emily', 'Howard', 'Bernadette', 'Like this', 'Bernadette', 'Howard', 'Raj', 'Mrs Wolowitz', 'Scene', 'Leonard', 'Amy', 'Sheldon', 'Penny', 'Emily', 'Man', 'Director', 'Wil Wheaton', 'Wil', 'Foreign Nurse', 'Like this', 'Ira Flatow', 'Sheldon', 'Ira', 'Scene', 'Amy', 'Leonard', 'Penny', 'Bernadette', 'Howard', 'Wil Wheaton', 'Wil', 'Raj', 'Bermadette', 'Barry', 'Like this', 'Penny', 'Sheldon', 'Leonard', 'Scene', 'Raj', 'Howard', 'Stuart', 'Mrs Koothrappali', 'Dr Koothrappali', 'Howard’s Mother', 'Like this', 'Leonard', 'Sheldon', 'Scene', 'Howard', 'Penny', 'Sheldon-bot', 'Raj', 'Steve Wozniak', 'Like this']
x:  ['The stairwell', 'The upstairs landing of Mrs Wolowitz’s house', 'A radio studio', 'The apartment', 'The apartment. Sheldon has a series of whiteboards across the room']
y:  ['Mrs Koothrappali', 'Like this', 'Scene', 'Penny', 'Mrs Wolowitz', 'Howard', 'Sheldon-bot', 'Howard’s Mother', 'Foreign Nurse', 'Wil Whe

In [0]:
from numpy import random
from pyecharts import HeatMap

x_axis = ["12a", "1a", "2a", "3a", "4a", "5a", "6a", "7a", "8a", "9a", "10a", "11a",
          "12p", "1p", "2p", "3p", "4p", "5p", "6p", "7p", "8p", "9p", "10p", "11p"]
y_aixs = ["Saturday", "Friday", "Thursday", "Wednesday", "Tuesday", "Monday", "Sunday"]
data = [[i, j, random.randint(0, 50)] for i in range(24) for j in range(7)]
heatmap = HeatMap()
heatmap.add("热力图直角坐标系", x_axis, y_aixs, data, is_visualmap=True,
            visual_text_color="#000", visual_orient='horizontal')

heatmap

## 情感分析
- SnowNLP是python中用来处理文本内容的，可以用来分词、标注、文本情感分析等，情感分析是简单的将文本分为两类，积极和消极，返回值为情绪的概率，越接近1为积极，接近0为消极

In [0]:
import numpy as np
from snownlp import SnowNLP
import matplotlib.pyplot as plt
 
f = open('comment.txt', 'r', encoding='UTF-8')
list = f.readlines()
sentimentslist = []
for i in list:
    s = SnowNLP(i)
    # print s.sentiments
    sentimentslist.append(s.sentiments)
plt.hist(sentimentslist, bins=np.arange(0, 1, 0.01), facecolor='g')
plt.xlabel('Sentiments Probability')
plt.ylabel('Quantity')
plt.title('Analysis of Sentiments')
plt.show()

In [0]:
from pyecharts import Parallel

schema = ["data", "AQI", "PM2.5", "PM10", "CO", "NO2"]
data = [
        [1, 91, 45, 125, 0.82, 34],
        [2, 65, 27, 78, 0.86, 45,],
        [3, 83, 60, 84, 1.09, 73],
        [4, 109, 81, 121, 1.28, 68],
        [5, 106, 77, 114, 1.07, 55],
        [6, 109, 81, 121, 1.28, 68],
        [7, 106, 77, 114, 1.07, 55],
        [8, 89, 65, 78, 0.86, 51, 26],
        [9, 53, 33, 47, 0.64, 50, 17],
        [10, 80, 55, 80, 1.01, 75, 24],
        [11, 117, 81, 124, 1.03, 45]
]
parallel = Parallel("平行坐标系-默认指示器")
parallel.config(schema) 
parallel.add("parallel", data, is_random=True)

parallel